In [63]:
import numpy as np
import pandas as pd

# Setting Random Seed for Reproducability
np.random.seed(42)

#Number of Samples
n_samples = 1000

# True Weights and bias(ground truth)
true_w1 = 12
true_w2 = 6
true_w3 = 2
true_bias = 20

#Generate Random Data for Features
employee_id = np.array([f"EMP_{i:03}" for i in range(1, n_samples+1)])
performance = np.random.randint(1,11, n_samples) # Performance: 1 to 10
years_of_experience = np.random.randint(1,11,n_samples)
projects_completed = years_of_experience + np.random.randint(1,4, n_samples)

# Calculate the target variable(bonus) using the true weights and bias
bonus = (true_w1 * performance) + (true_w2 * years_of_experience) + (true_w3 * projects_completed) + true_bias

# Create a data Frame

df =  pd.DataFrame({
    'employee_id' : employee_id,
    'performance' : performance,
    'years_of_experience' : years_of_experience,
    'projects_completed' : projects_completed,
    'bonus' : bonus
})

df.to_csv(r"C:\Users\Sumit Sah\ML Course\deep-learning-journey\datasets\employee_bonus_data.csv", index=False)

In [49]:
df.head()

,employee_id,performance,years_of_experience,projects_completed,bonus
0,EMP_001,7,1,3,116
1,EMP_002,4,8,10,136
2,EMP_003,8,4,5,150
3,EMP_004,5,4,7,118
4,EMP_005,7,5,6,146


In [50]:
df.shape

(1000, 5)

## Building Neural Network using PyTorch

In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [52]:
X = df[['performance', 'years_of_experience','projects_completed']].values
y = df[['bonus']].values

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42)

In [53]:
X_train.shape

(800, 3)

In [54]:
X_test.shape

(200, 3)

In [55]:
X_train_tensor = torch.tensor(X_train, dtype = torch.float32)
y_train_tensor = torch.tensor(y_train, dtype = torch.float32)
X_test_tensor = torch.tensor(X_test, dtype = torch.float32)
y_test_tensor = torch.tensor(y_test, dtype = torch.float32)

In [56]:
class BonusPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(3,1)
            
        )
    def forward(self, x):
        return self.network(x)

model = BonusPredictor()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.005)

epochs = 5000

for epoch in range(epochs):
    # 1. Forward Pass
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)

    # 2. Backward Pass
    optimizer.zero_grad()
    loss.backward()

    # 3. Update Weights
    optimizer.step()

    #print loss in every 100 epochs
    if ((epoch + 1) % 100) == 0:
        print(f"Epoch[{epoch + 1}/{epochs}] -> Loss: {loss.item() :.2f}")
    

Epoch[100/5000] -> Loss: 22.21
Epoch[200/5000] -> Loss: 19.27
Epoch[300/5000] -> Loss: 16.86
Epoch[400/5000] -> Loss: 14.80
Epoch[500/5000] -> Loss: 13.00
Epoch[600/5000] -> Loss: 11.43
Epoch[700/5000] -> Loss: 10.06
Epoch[800/5000] -> Loss: 8.85
Epoch[900/5000] -> Loss: 7.78
Epoch[1000/5000] -> Loss: 6.84
Epoch[1100/5000] -> Loss: 6.02
Epoch[1200/5000] -> Loss: 5.30
Epoch[1300/5000] -> Loss: 4.66
Epoch[1400/5000] -> Loss: 4.10
Epoch[1500/5000] -> Loss: 3.61
Epoch[1600/5000] -> Loss: 3.17
Epoch[1700/5000] -> Loss: 2.79
Epoch[1800/5000] -> Loss: 2.45
Epoch[1900/5000] -> Loss: 2.16
Epoch[2000/5000] -> Loss: 1.90
Epoch[2100/5000] -> Loss: 1.67
Epoch[2200/5000] -> Loss: 1.47
Epoch[2300/5000] -> Loss: 1.29
Epoch[2400/5000] -> Loss: 1.14
Epoch[2500/5000] -> Loss: 1.00
Epoch[2600/5000] -> Loss: 0.88
Epoch[2700/5000] -> Loss: 0.77
Epoch[2800/5000] -> Loss: 0.68
Epoch[2900/5000] -> Loss: 0.60
Epoch[3000/5000] -> Loss: 0.53
Epoch[3100/5000] -> Loss: 0.46
Epoch[3200/5000] -> Loss: 0.41
Epoch[3300

In [57]:
model.eval()

with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
print(f"Test Loss : {test_loss.item(): 0.2f}")

Test Loss :  0.04


In [58]:
test_predictions[:3]

tensor([[ 91.8062],
        [130.0900],
        [ 85.6190]])

In [59]:
y_test_tensor[:3]

tensor([[ 92.],
        [130.],
        [ 86.]])

In [61]:
for name, param in model.named_parameters():
    print(name, param.data)

network.0.weight tensor([[12.0336,  5.8464,  2.1832]])
network.0.bias tensor([19.2426])
